## From chatGPT (experimental)

### 💡 **Recommended Approach** (Latent Gaussian Graphical Model):

To reliably induce correlation across edges:

- **Step 1**: Sample a latent Gaussian vector for each gene with a structured covariance.
- **Step 2**: Introduce correlation explicitly through the graph structure.
- **Step 3**: Convert latent Gaussian vectors to integer counts via a suitable link function (Poisson).

---

## **Minimal Implementation (no spatial distances):**

Here's how to achieve this step-by-step clearly:

### **1. Define Covariance using Graph Structure**:

Given adjacency `G` (edges), we construct covariance as:


$$C = (\alpha I + \beta G)^{-1}$$

where:

- \(G\) is your adjacency matrix (binary, symmetric),
- \(\alpha\) (self-weight) ensures positive definiteness (e.g. α = degree + small constant),
- \(\beta\) controls edge-based correlation strength.

### **2. Generate Two Latent Gaussian Vectors**:

We want counts for two genes, say ligand (`L`) and receptor (`R`).  
To induce correlation specifically over edges, we sample latent Gaussian vectors jointly with covariance:

$$
\text{Cov}\bigl([L, R]\bigr) = 
\begin{pmatrix}
C & \rho C \\
\rho C & C
\end{pmatrix}
$$

where \(\rho\) controls desired correlation.

### **3. Convert Latent Variables to Counts**:

Apply exponential link to Poisson rates:

$$
\lambda_L = \exp(\mu_L + Z_L), \quad \lambda_R = \exp(\mu_R + Z_R)
$$

and then sample:

- \( L_i \sim \text{Poisson}(\lambda_{L,i}) \)
- \( R_i \sim \text{Poisson}(\lambda_{R,i}) \)

---

In [2]:
import numpy as np
from scipy.stats import multivariate_normal, poisson
from scipy.sparse import csgraph
from numpy.linalg import inv

In [3]:
def simulate_counts_graph_corr(G_adj, desired_corr=0.8, mu_L=1.0, mu_R=1.0):
    """
    Simulate counts for two genes with correlation across edges.

    Parameters:
        G_adj (np.ndarray): adjacency matrix (binary, symmetric)
        desired_corr (float): target correlation between genes across edges
        mu_L, mu_R (float): baseline log-expression for L and R genes

    Returns:
        L_counts, R_counts (np.ndarray): integer counts per node
    """
    n = G_adj.shape[0]

    # Ensure G_adj symmetric and binary
    G_adj = (G_adj > 0).astype(float)
    np.fill_diagonal(G_adj, 0)

    # Degree matrix and small epsilon to ensure positive definiteness
    degrees = G_adj.sum(axis=1)
    alpha = np.diag(degrees + 0.1)
    beta = 0.9  # strength of correlation between neighbors (tunable)

    # Precision and covariance matrix (graph-structured covariance)
    precision_single = alpha - beta * G_adj
    covariance_single = inv(precision_single)

    # Joint covariance matrix (2 genes: L and R)
    covariance_joint = np.block([
        [covariance_single, desired_corr * covariance_single],
        [desired_corr * covariance_single, covariance_single]
    ])

    # Sample latent Gaussian vectors
    latent = multivariate_normal.rvs(
        mean=np.zeros(2*n), cov=covariance_joint
    )
    Z_L, Z_R = latent[:n], latent[n:]

    # Convert latent Gaussian to Poisson rates
    lambda_L = np.exp(mu_L + Z_L)
    lambda_R = np.exp(mu_R + Z_R)

    # Sample counts
    L_counts = poisson.rvs(lambda_L)
    R_counts = poisson.rvs(lambda_R)

    return L_counts, R_counts